In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from pathlib import Path
import json

# Step 1: Define paths and hyperparameters
DATA_ROOT = "/content/drive/MyDrive/NTU-Roselab-Dataset"
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 8
LEARNING_RATE = 1e-4
N_FOLDS = 5
MODEL_NAME = "Fourier_MobileNetV2"
OUTPUT_DIR = f"/content/drive/MyDrive/Recapture_Photo_Detection/{MODEL_NAME}/results"
SPLIT_DIR = "/content/drive/MyDrive/Recapture_Photo_Detection"
PREPROCESSING = "Fourier"
HYPERPARAMETERS = {
    "learning_rate": LEARNING_RATE,
    "batch_size": BATCH_SIZE,
    "optimizer": "Adam",
    "epochs": EPOCHS,
    "n_folds": N_FOLDS,
    "dropout_rate": 0.4
}

# Step 2: Mount Google Drive and verify dataset path
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except ImportError:
    raise ImportError("This script must be run in Google Colab with Google Drive mounted.")
if not os.path.exists(DATA_ROOT):
    raise FileNotFoundError(f"Dataset directory {DATA_ROOT} does not exist. Please check the path.")

# Step 3: Check dataset balance
def check_dataset_balance(data_root):
    originals_path = os.path.join(data_root, 'originals')
    recaptures_path = os.path.join(data_root, 'recaptures')
    originals_count = sum(len(files) for _, _, files in os.walk(originals_path))
    recaptures_count = sum(len(files) for _, _, files in os.walk(recaptures_path))
    print(f"Dataset Balance: {originals_count} originals, {recaptures_count} recaptures")
    return originals_count, recaptures_count

originals_count, recaptures_count = check_dataset_balance(DATA_ROOT)

# Step 4: Define Hann window function
@tf.function
def hann2d(h, w):
    hann1 = tf.signal.hann_window(h, periodic=True)
    hann2 = tf.signal.hann_window(w, periodic=True)
    return tf.sqrt(tf.tensordot(hann1, hann2, axes=0))

# Step 5: Define FFT preprocessing function
@tf.function
def fourier_preprocess(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    gray = tf.image.rgb_to_grayscale(img)
    gray = tf.cast(gray, tf.float32)
    gray -= tf.reduce_mean(gray)
    hann_window = hann2d(IMG_SIZE, IMG_SIZE)
    gray *= hann_window
    fft = tf.signal.fft2d(tf.complex(gray[..., 0], 0.0))
    fft = tf.signal.fftshift(fft)
    spectrum = tf.math.log1p(tf.abs(fft))
    spectrum = (spectrum - tf.reduce_mean(spectrum)) / (tf.math.reduce_std(spectrum) + 1e-6)
    spectrum = (spectrum - tf.reduce_min(spectrum)) / (tf.reduce_max(spectrum) - tf.reduce_min(spectrum) + 1e-6)
    combined = tf.stack([spectrum, spectrum, spectrum], axis=-1)
    return combined, label

# Step 6: Define custom rotation function
@tf.function
def random_rotation(img, max_angle=0.1):  # max_angle in radians (~10 degrees)
    angles = [0, np.pi/2, np.pi, 3*np.pi/2]  # 0°, 90°, 180°, 270°
    k = tf.random.uniform(shape=(), minval=0, maxval=len(angles), dtype=tf.int32)
    img = tf.image.rot90(img, k)
    return img

# Step 7: Define data augmentation
@tf.function
def augment_image(img, label):
    img = tf.image.random_flip_left_right(img)
    img = random_rotation(img, max_angle=0.1)
    return img, label

# Step 8: Load and split dataset (single train-test split)
dataset = image_dataset_from_directory(
    DATA_ROOT,
    labels='inferred',
    label_mode='binary',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

# Convert dataset to NumPy arrays for cross-validation
images, labels = [], []
for img_batch, label_batch in dataset:
    images.append(img_batch.numpy())
    labels.append(label_batch.numpy())
images = np.concatenate(images, axis=0)
labels = np.concatenate(labels, axis=0).flatten()

# Split into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, stratify=labels, random_state=42
)

# Create directory for train-test split
Path(SPLIT_DIR).mkdir(parents=True, exist_ok=True)

# Save train-test split for consistency across models
np.save(os.path.join(SPLIT_DIR, 'X_train.npy'), X_train)
np.save(os.path.join(SPLIT_DIR, 'X_test.npy'), X_test)
np.save(os.path.join(SPLIT_DIR, 'y_train.npy'), y_train)
np.save(os.path.join(SPLIT_DIR, 'y_test.npy'), y_test)

# Create test dataset
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE).map(fourier_preprocess).prefetch(tf.data.AUTOTUNE)

# Step 9: Define function to create MobileNetV2 model
def create_model():
    base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
    for layer in base.layers[:-20]:
        layer.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(HYPERPARAMETERS['dropout_rate'])(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(base.input, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 10: Convert NumPy types to JSON-serializable types
def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# Step 11: Define function to save results
def save_model_results(model, dataset, history, model_name, output_dir, fold=None, preprocessing='None', hyperparameters=None):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    fold_str = f"_fold_{fold}" if fold is not None else ""

    # Evaluate on dataset
    y_true, y_pred = [], []
    for imgs, labels in dataset:
        preds = (model.predict(imgs, verbose=0) > 0.5).astype(int)
        y_true.extend(labels.numpy().astype(int))
        y_pred.extend(preds.flatten())

    # Check prediction distribution
    originals_pred = sum(1 for p in y_pred if p == 0)
    recaptures_pred = sum(1 for p in y_pred if p == 1)
    print(f"Predictions {fold_str}: {originals_pred} originals, {recaptures_pred} recaptures")

    # Classification report
    class_report = classification_report(y_true, y_pred, target_names=['originals', 'recaptured'], output_dict=True)
    class_report_df = pd.DataFrame(class_report).transpose()
    class_report_df.to_csv(f'{output_dir}/{model_name}_classification_report{fold_str}.csv')

    # Confusion matrix (detailed)
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['originals', 'recaptured'], yticklabels=['originals', 'recaptured'])
    plt.title(f'Confusion Matrix - {model_name}{fold_str}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(f'{output_dir}/{model_name}_confusion_matrix{fold_str}.png')
    plt.close()

    # Save confusion matrix as CSV
    cm_df = pd.DataFrame(cm, index=['True_originals', 'True_recaptured'], columns=['Pred_originals', 'Pred_recaptured'])
    cm_df.to_csv(f'{output_dir}/{model_name}_confusion_matrix{fold_str}.csv')

    # Model summary (only for final model)
    if fold is None:
        summary_file = f'{output_dir}/{model_name}_summary.txt'
        with open(summary_file, 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))

    # Calculate total and trainable parameters
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])

    # Aggregate results
    results = {
        'Model': model_name,
        'Preprocessing': preprocessing,
        'Accuracy': class_report['accuracy'],
        'Total_Parameters': total_params,
        'Trainable_Parameters': trainable_params,
        'Fold': fold if fold is not None else 'Final'
    }
    if hyperparameters:
        results.update(hyperparameters)
    for label, metrics in class_report.items():
        if isinstance(metrics, dict):
            results.update({
                f'Precision_{label}': metrics['precision'],
                f'Recall_{label}': metrics['recall'],
                f'F1-Score_{label}': metrics['f1-score'],
                f'Support_{label}': metrics['support']
            })

    # Convert NumPy types to JSON-serializable types
    results = {k: convert_to_serializable(v) for k, v in results.items()}

    # Save results to JSON
    with open(f'{output_dir}/{model_name}_results{fold_str}.json', 'w') as f:
        json.dump(results, f, indent=4)

    # Plot and save accuracy/loss curves
    if history is not None:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Accuracy Curve - {model_name}{fold_str}')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Loss Curve - {model_name}{fold_str}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/{model_name}_accuracy_loss_curve{fold_str}.png')
        plt.close()

    # Save model weights (only for final model)
    if fold is None:
        model.save(f'{output_dir}/{model_name}_model.h5')

    return results

# Step 12: Perform 5-fold cross-validation
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
fold_results = []
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    print(f"\nTraining Fold {fold + 1}/{N_FOLDS}")

    # Create train and validation datasets for this fold
    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

    train_ds = tf.data.Dataset.from_tensor_slices((X_fold_train, y_fold_train)).batch(BATCH_SIZE).map(fourier_preprocess).map(augment_image).prefetch(tf.data.AUTOTUNE)
    val_ds = tf.data.Dataset.from_tensor_slices((X_fold_val, y_fold_val)).batch(BATCH_SIZE).map(fourier_preprocess).prefetch(tf.data.AUTOTUNE)

    # Create and train model with early stopping
    model = create_model()
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[early_stopping], verbose=1)

    # Save results for this fold
    results = save_model_results(
        model, val_ds, history, MODEL_NAME, OUTPUT_DIR,
        fold=fold + 1, preprocessing=PREPROCESSING, hyperparameters=HYPERPARAMETERS
    )
    fold_results.append(results)

# Step 13: Train final model on full training set
print("\nTraining final model on full training set")
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE).map(fourier_preprocess).map(augment_image).prefetch(tf.data.AUTOTUNE)
model = create_model()
history = model.fit(train_ds, epochs=EPOCHS, verbose=1)

# Step 14: Evaluate final model on test set and save results
results = save_model_results(
    model, test_ds, history, MODEL_NAME, OUTPUT_DIR,
    preprocessing=PREPROCESSING, hyperparameters=HYPERPARAMETERS
)
print(f"Final Results for {MODEL_NAME}:", results)

# Step 15: Aggregate cross-validation results
if fold_results:
    fold_df = pd.DataFrame(fold_results)
    mean_results = {
        'Model': MODEL_NAME,
        'Preprocessing': PREPROCESSING,
        'Mean_Accuracy': fold_df['Accuracy'].mean(),
        'Std_Accuracy': fold_df['Accuracy'].std(),
        'Mean_Precision_recaptured': fold_df['Precision_recaptured'].mean(),
        'Mean_Recall_recaptured': fold_df['Recall_recaptured'].mean(),
        'Mean_F1-Score_recaptured': fold_df['F1-Score_recaptured'].mean(),
        'Mean_Total_Parameters': fold_df['Total_Parameters'].mean(),
        'Mean_Trainable_Parameters': fold_df['Trainable_Parameters'].mean()
    }
    # Convert NumPy types in mean_results
    mean_results = {k: convert_to_serializable(v) for k, v in mean_results.items()}
    with open(f'{OUTPUT_DIR}/{MODEL_NAME}_cv_summary.json', 'w') as f:
        json.dump(mean_results, f, indent=4)
    fold_df.to_csv(f'{OUTPUT_DIR}/{MODEL_NAME}_cv_results.csv', index=False)
    print("\nCross-Validation Summary:", mean_results)

Mounted at /content/drive
Dataset Balance: 1211 originals, 1199 recaptures
Found 2401 files belonging to 2 classes.

Training Fold 1/5


/tmp/ipython-input-2269862735.py:134: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 88s 586ms/step - accuracy: 0.4661 - loss: 0.7391 - val_accuracy: 0.5000 - val_loss: 0.8244
Epoch 2/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 50s 519ms/step - accuracy: 0.4900 - loss: 0.7186 - val_accuracy: 0.5000 - val_loss: 0.7667
Epoch 3/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 50s 517ms/step - accuracy: 0.5373 - loss: 0.7025 - val_accuracy: 0.5000 - val_loss: 0.7485
Epoch 4/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 82s 522ms/step - accuracy: 0.5002 - loss: 0.7071 - val_accuracy: 0.5000 - val_loss: 0.7928
Epoch 5/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 82s 519ms/step - accuracy: 0.4973 - loss: 0.7050 - val_accuracy: 0.5000 - val_loss: 0.7004
Epoch 6/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 51s 532ms/step - accuracy: 0.4963 - loss: 0.7002 - val_accuracy: 0.5000 - val_loss: 0.7525
Epoch 7/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 50s 514ms/step - accuracy: 0.4962 - loss: 0.7014 - val_accuracy: 0.5000 - val_loss: 0.7538
Epoch 8/8
96/96 ━━━━━━━━━━━━━━━━━━━━ 83s 515ms/step - a

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Training Fold 2/5


...


Training final model on full training set


/tmp/ipython-input-2269862735.py:134: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))


Epoch 1/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 67s 401ms/step - accuracy: 0.5084 - loss: 0.7144
Epoch 2/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 51s 418ms/step - accuracy: 0.5062 - loss: 0.7109
Epoch 3/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 52s 431ms/step - accuracy: 0.5139 - loss: 0.7102
Epoch 4/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 80s 415ms/step - accuracy: 0.4935 - loss: 0.7021
Epoch 5/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 50s 413ms/step - accuracy: 0.5083 - loss: 0.7008
Epoch 6/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 83s 418ms/step - accuracy: 0.4963 - loss: 0.6977
Epoch 7/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 82s 419ms/step - accuracy: 0.4904 - loss: 0.6986
Epoch 8/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 51s 423ms/step - accuracy: 0.4942 - loss: 0.6942
Predictions : 0 originals, 481 recaptures


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Final Results for Fourier_MobileNetV2: {'Model': 'Fourier_MobileNetV2', 'Preprocessing': 'Fourier', 'Accuracy': 0.498960498960499, 'Total_Parameters': 2422081, 'Trainable_Parameters': 1370177, 'Fold': 'Final', 'learning_rate': 0.0001, 'batch_size': 16, 'optimizer': 'Adam', 'epochs': 8, 'n_folds': 5, 'dropout_rate': 0.4, 'Precision_originals': 0.0, 'Recall_originals': 0.0, 'F1-Score_originals': 0.0, 'Support_originals': 241.0, 'Precision_recaptured': 0.498960498960499, 'Recall_recaptured': 1.0, 'F1-Score_recaptured': 0.665742024965326, 'Support_recaptured': 240.0, 'Precision_macro avg': 0.2494802494802495, 'Recall_macro avg': 0.5, 'F1-Score_macro avg': 0.332871012482663, 'Support_macro avg': 481.0, 'Precision_weighted avg': 0.2489615795229101, 'Recall_weighted avg': 0.498960498960499, 'F1-Score_weighted avg': 0.332178972955672, 'Support_weighted avg': 481.0}

Cross-Validation Summary: {'Model': 'Fourier_MobileNetV2', 'Preprocessing': 'Fourier', 'Mean_Accuracy': 0.4994791666666667, 'Std_